In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import scipy as sp
import scipy.stats as stats
import statsmodels.formula.api as smf

In [2]:
hmd = pd.read_csv("./hmd_weekly_deaths/stmf.csv") #mortality dataset
coviddb = pd.read_csv("owid-covid-data.csv") #covid dataset

#Make a list of the country codes from the coviddb that appear in the hmd
country_codes = ['AUS', 'AUT', 'BEL', 'BGR', 'HRV', 'CAN', 'CHL', 'CZE', 'DNK', 'GBR', 'EST',
 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'ISL', 'ISR', 'ITA', 'LVA', 'LTU', 'LUX', 'NLD',
  'NZL', 'NOR', 'POL', 'PRT', 'KOR', 'RUS', 'SVN', 'SVK', 'ESP', 'CHE', 'SWE', 'TWN', 'USA']

coviddb_selected = coviddb[coviddb.iso_code.isin(country_codes)] #filter out unmatching countries

#Trim data to useful attributes
coviddb_selected=coviddb_selected[['iso_code','date', 'total_cases','new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'new_cases_per_million', 'hosp_patients', 'population']]

coviddb_selected['date']=pd.to_datetime(coviddb_selected['date']) #convert date column to datetime

#convert country to category
coviddb_selected.iso_code.astype('category')

#resample from daily to weekly
coviddb_selected = coviddb_selected.reset_index().groupby('iso_code', as_index=False).resample('w', on = 'date').\
agg({'total_cases':'first', 'new_cases':'sum','total_deaths':'first','new_deaths':'sum','total_cases_per_million':'first', 'new_cases_per_million':'sum',\
'hosp_patients':'first','population':'first'}).reset_index()

#restore columns
coviddb_selected.rename(columns={'level_0':'country'}, inplace = True)

In [3]:
#Country codes have changed to integers; this changes them back to country codes
coviddb_selected.country.astype('int64')
country_index = np.arange(0, 35, 1)
country_dict = dict(zip(country_index, country_codes))
coviddb_selected = coviddb_selected.replace({"country": country_dict})

In [4]:
week = coviddb_selected['date'].dt.week
year = coviddb_selected['date'].dt.year
coviddb_selected['week'] = week
coviddb_selected['year'] = year

C:\Users\melni\AppData\Local\Temp\ipykernel_11544\1723132619.py:1: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  week = coviddb_selected['date'].dt.week


In [5]:
#Reorder columns and remove original date
cols = ['country', 'date', 'year', 'week', 'total_cases', 'new_cases', 'total_deaths', 'new_deaths', 'total_cases_per_million', 'hosp_patients', 'population']
coviddb_selected = coviddb_selected[cols]

In [6]:
#Convert hmd country codes to match covid db and match column titles
country_conversion = { 'AUS2' : 'AUS', 'GBRTENW' : 'GBR', 'GBR_NIR' : 'GBR', 'GBR_SCO' : 'GBR', 'FRATNP' : 'FRA', 'DEUTNP' : 'DEU', 'NZL_NP' : 'NZL'}
hmd = hmd.replace({"CountryCode" : country_conversion})
hmd.rename(columns={'CountryCode' :'country', 'Year' : 'year', 'Week' : 'week'}, inplace = True)
hmd.country.astype('category')

0         AUS
1         AUS
2         AUS
3         AUS
4         AUS
         ... 
109255    USA
109256    USA
109257    USA
109258    USA
109259    USA
Name: country, Length: 109260, dtype: category
Categories (36, object): ['AUS', 'AUT', 'BEL', 'BGR', ..., 'SVN', 'SWE', 'TWN', 'USA']

In [7]:
#retain only the combined male and female deaths
hmd= hmd[hmd.Sex == 'b']

In [8]:
#merge various Great Britain rows into one
hmd_selected = hmd.groupby(['country', 'week', 'year']).sum().reset_index()

In [9]:
#Trim excess columns
hmd = hmd_selected[['country', 'week', 'year', 'DTotal']]

In [10]:
#adding a moving average of total deaths... 
hmd['date'] = pd.to_datetime(hmd.week.astype(str) + hmd.year.astype(str).add('-7') ,format='%V%G-%u')
#hmd['date'] = pd.to_datetime(hmd['date'] + ' 0', format='%Y %W %w')
hmd = hmd.sort_values(by='date')
hmd['mov_avg'] = hmd.groupby('country')['DTotal'].transform(lambda x: x.rolling(7, 1).mean())

C:\Users\melni\AppData\Local\Temp\ipykernel_11544\2752251486.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hmd['date'] = pd.to_datetime(hmd.week.astype(str) + hmd.year.astype(str).add('-7') ,format='%V%G-%u')


In [11]:
#add years relative to 2020 for regression to predict deaths without including COVID
hmd['years_relative'] = hmd['date'].dt.year - 2020
#Drop Great Britain data prior to 2015--only 2015 on includes all four UK countries
hmd.drop(hmd[(hmd.country == 'GBR') & (hmd.year < 2015)].index, inplace = True)

In [12]:
#Perform linear regression to predict moving average of weekly deaths based on year (pre-2020)
for country in country_codes:
  for i in range(1, 53):
    training_set = hmd[(hmd.country == country) & (hmd.years_relative < 0) & (hmd.week == i)]
    model =  smf.ols(formula='mov_avg~years_relative', data=training_set).fit()
    hmd.loc[(hmd.country == country) & (hmd.week == i), 'pred_deaths'] = model.predict(hmd[(hmd.country == country) & (hmd.week == i)])

In [13]:
#Compute excess mortality
hmd['excess_mortality'] = hmd['mov_avg'] - hmd['pred_deaths']

In [14]:
#create a dictionary that translates country into population
pop_dict = coviddb_selected.groupby('country')['population'].first().to_dict()
#Use population to compute excess mortality per capita
hmd['em_per_capita'] = hmd['excess_mortality'] / hmd['country'].map(pop_dict)

In [15]:
#perform min-max normalization
max_em_per_capita = hmd.em_per_capita.max()
min_em_per_capita = hmd.em_per_capita.min()
hmd['normalized_em'] = (hmd['em_per_capita'] - min_em_per_capita) / (max_em_per_capita - min_em_per_capita)

In [16]:
#compute rolling average for new covid cases
covid = coviddb_selected
covid['mov_avg_cases'] = covid.groupby('country')['new_cases'].transform(lambda x: x.rolling(7, 1).mean())
covid['new_cases_per_capita'] = covid['mov_avg_cases'] / covid['population']
#and normalize
max_cases_per_capita = covid.new_cases_per_capita.max()
min_cases_per_capita = covid.new_cases_per_capita.min()
covid['normalized_cases'] = (covid['new_cases_per_capita'] - min_cases_per_capita) / (max_cases_per_capita - min_cases_per_capita)

In [17]:
#merge datasets
merged_df = hmd.merge(covid, how='left', on=['country', 'date'])
merged_df = merged_df.sort_values(by='date')

In [18]:
merged_df.rename(columns={'week_x':'week', 'year_x' : 'year'}, inplace = True)
merged_df = merged_df.drop(['year_y', 'week_y'], axis=1)

In [19]:
merged_df.to_csv('clean_combined.csv', index=False)

In [20]:
#Drop ITA due to extreme outlier aspect and renormalize
drop_ITA = pd.read_csv('clean_combined.csv', parse_dates = ['date'])
drop_ITA = drop_ITA[drop_ITA.country != 'ITA']
ax_em_per_capita = drop_ITA.em_per_capita.max()
min_em_per_capita = drop_ITA.em_per_capita.min()
print(max_em_per_capita, min_em_per_capita)
drop_ITA['normalized_em'] = (drop_ITA['em_per_capita'] - min_em_per_capita) / (max_em_per_capita - min_em_per_capita)
drop_ITA.to_csv('clean_combined_no_ITA.csv', index=False)

0.017982702944009037 -0.0002590487422515
